In [54]:
%load_ext autoreload
%autoreload 2
import logging
logging.basicConfig(level=logging.INFO)
import random
random.seed(0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
from src.tradecli import *
from src.utils import *
from src.journal import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [56]:
display(full_df)

,type_1_m5,type_1_m1,type_1_m30,type_1_h4,type_1_m15,risk_reward_ratio,type_1_h1,type_1_d,type_1_M,type_1_w,has_type_1,has_type_2,has_type_3,has_time_frame
0,False,True,False,False,False,2.000000,False,False,False,False,True,False,False,False
1,False,False,False,False,False,0.666667,False,False,False,False,False,False,False,False
2,False,False,False,False,False,10.000000,False,False,False,False,False,False,False,False
3,False,False,False,False,False,10.000000,False,False,False,False,False,False,False,False
4,False,True,False,False,False,5.000000,False,False,False,False,True,False,False,False
5,False,False,False,False,False,5.913580,False,False,True,False,True,False,False,False
6,True,False,False,False,False,7.482759,False,False,False,False,True,False,False,False
7,False,False,False,False,True,2.215190,False,False,False,False,True,False,False,False
8,False,False,False,False,False,3.978022,False,False,True,False,True,False,False,False
9,False,False,False,False,True,1.348315,False,False,False,False,True,False,False,False


In [57]:
full_df.dtypes

type_1_m5               bool
type_1_m1               bool
type_1_m30              bool
type_1_h4               bool
type_1_m15              bool
risk_reward_ratio    float64
type_1_h1               bool
type_1_d                bool
type_1_M                bool
type_1_w                bool
has_type_1              bool
has_type_2              bool
has_type_3              bool
has_time_frame          bool
dtype: object

In [58]:
!rm -rf output

In [85]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
j.to_markdown(output_dir='output')

INFO:root:Converting trades to markdown
INFO:root:Files for trades found
INFO:root:Files Found #: 14, Total Trades #: 14max files for trade: 0min files for trade: 0avg files for trade: 0.0
INFO:root:Descriptive Statistics:           0
count  14.0
mean    0.0
std     0.0
min     0.0
25%     0.0
50%     0.0
75%     0.0
max     0.0
INFO:root:Writing index markdown
INFO:root:Calculating simple statistics


INFO:root:Analyzing tag relevance
INFO:root:Finding best single tags and subsets
INFO:root:Finding best single tags
INFO:root:Finding best tag subsets
INFO:root:Finding best tag subsets
INFO:root:Selecting top 10 features using RFE
INFO:root:Dropped 6 rows with NaN in 'outcome' y variable
INFO:root:X features # 41: {'type_3_m1', 'type_3_M', 'htf_poi_ltf_confirmation_False', 'type_2_m30', 'type_1_m5', 'type_1_m1', 'type_3_h4', 'type_3_w', 'htf_poi_ltf_confirmation', 'type_2_m15', 'type_3_m15', 'type_2_h4', 'close_price', 'type_1_m30', 'management_strategy', 'outcome_loss', 'type_3_d', 'type_2_M', 'type_2_m1', 'type_2_w', 'htf_poi_ltf_confirmation_True', 'type_3_h1', 'type_1_h4', 'type_2_m5', 'management_strategy_be_after_push', 'type_2_h1', 'outcome_win', 'type_1_m15', 'risk_reward_ratio', 'type_1_h1', 'management_strategy_be_after_1r', 'type_1_d', 'type_1_M', 'session', 'type_3_m5', 'type_3_m30', 'type_2_d', 'management_strategy_strategy_2', 'type_1_w', 'side', 'management_strategy_no_

In [105]:
config = Config(
    include_patterns=["outcome"],
    exclude_patterns=[],
    force_values={},
)

df = j.to_dataframe(config=)

In [106]:
df

,trade_uid,type_3_m1,type_3_M,numerical_confidence,outcome,type_1_m5,type_2_m30,type_1_m1,type_3_h4,type_3_w,htf_poi_ltf_confirmation,type_2_m15,type_3_m15,type_2_h4,return,close_price,type_1_m30,management_strategy,potential_risk_reward,initial_risk_reward,entry_price,type_3_d,type_2_M,type_2_m1,type_2_w,entry_time,potential_price,potential_return,SL_distance,type_3_h1,type_1_h4,type_2_m5,type_2_h1,type_1_m15,risk_reward_ratio,tp_price,type_1_h1,type_1_d,sl_price,type_1_M,session,initial_return,type_3_m5,type_3_m30,type_2_d,type_1_w,side,TP_distance,htf_poi_ltf_confirmation_False,htf_poi_ltf_confirmation_True,management_strategy_be_after_1r,management_strategy_be_after_push,management_strategy_no_management,management_strategy_strategy_2,outcome_loss,outcome_win
0,1,False,False,3.0,NaN,False,False,True,False,False,False,False,True,False,NaN,NaN,False,no_management,NaN,2.000000,1.1000,False,False,False,False,2025-01-02 02:24:55.600575,NaN,NaN,0.0050,False,False,False,False,False,2.000000,1.1100,False,False,1.0950,False,tokyo,0.0100,False,False,False,False,long,0.0100,True,False,False,False,True,False,False,False
1,2,False,False,2.0,loss,False,False,False,False,False,True,False,False,False,-0.0145,1.1855,False,no_management,NaN,0.666667,1.2000,False,False,False,False,2025-01-02 02:24:55.601552,NaN,NaN,0.0150,False,False,False,True,False,0.666667,1.2100,False,False,1.1850,False,tokyo,0.0100,False,False,False,False,long,0.0100,False,True,False,False,True,False,True,False
2,3,False,False,NaN,NaN,False,False,False,False,False,False,False,False,False,NaN,NaN,False,no_management,NaN,10.000000,1.3000,False,False,False,False,2024-12-27 10:00:01.000000,NaN,NaN,0.5000,False,False,False,False,False,10.000000,1.3300,False,False,1.2970,False,london,0.0300,False,False,False,False,long,0.0300,True,False,False,False,True,False,False,False
3,4,False,False,4.0,win,False,False,False,False,False,False,False,False,False,0.0050,1.3050,False,strategy_2,5.000000,10.000000,1.3000,False,False,False,False,2024-12-25 10:00:00.000000,1.3150,0.0150,0.5000,False,False,False,False,False,10.000000,1.3300,False,False,1.2970,False,london,0.0300,False,False,False,False,long,0.0300,True,False,False,False,False,True,False,True
4,5,False,False,3.0,win,False,False,False,False,False,False,False,False,False,0.0132,1.1248,False,be_after_1r,3.397959,3.000000,1.1116,False,False,False,False,2025-01-02 02:24:55.604629,1.1449,0.0333,0.0098,False,False,False,False,False,3.000000,1.1410,False,True,1.1018,False,tokyo,0.0294,False,False,True,False,long,0.0294,True,False,True,False,False,False,False,True
5,6,False,False,3.0,NaN,False,False,False,False,False,True,True,False,False,NaN,NaN,False,no_management,1.590909,3.943182,1.1213,False,False,False,False,2025-01-02 02:24:55.605254,1.1353,0.0140,0.0088,False,False,False,False,False,3.943182,1.1560,False,False,1.1125,True,tokyo,0.0347,False,False,False,False,long,0.0347,False,True,False,False,True,False,False,False
6,7,False,False,4.0,loss,False,False,False,False,False,False,False,False,False,-0.0118,1.1206,False,be_after_push,6.558824,3.323529,1.1324,False,False,False,True,2025-01-02 02:24:55.605795,1.1770,0.0446,0.0068,False,True,False,False,False,3.323529,1.1550,False,False,1.1256,False,tokyo,0.0226,False,False,False,False,long,0.0226,True,False,False,True,False,False,True,False
7,8,False,False,5.0,win,False,False,True,False,False,True,False,False,False,0.0114,1.1580,False,be_after_push,5.400000,7.927273,1.1466,False,True,False,False,2025-01-02 02:24:55.606385,1.1763,0.0297,0.0055,False,False,False,False,False,7.927273,1.1902,False,False,1.1411,False,tokyo,0.0436,False,False,False,False,long,0.0436,False,True,False,True,False,False,False,True
8,9,False,False,5.0,NaN,False,False,False,False,False,True,False,False,True,NaN,NaN,True,no_management,5.264151,8.811321,1.1381,False,False,False,False,2025-01-02 02:24:55.606952,1.1660,0.0279,0.0053,False,False,False,False,False,8.811321,1.1848,False,False,1.1328,False,tokyo,0.0467,False,F

In [150]:
df:pd.DataFrame = j.to_dataframe(config=None)
df

,trade_uid,type_3_m1,type_3_M,numerical_confidence,outcome,type_1_m5,type_2_m30,type_1_m1,type_3_h4,type_3_w,htf_poi_ltf_confirmation,type_2_m15,type_3_m15,type_2_h4,return,close_price,type_1_m30,management_strategy,potential_risk_reward,initial_risk_reward,entry_price,type_3_d,type_2_M,type_2_m1,type_2_w,entry_time,potential_price,potential_return,SL_distance,type_3_h1,type_1_h4,type_2_m5,type_2_h1,type_1_m15,risk_reward_ratio,tp_price,type_1_h1,type_1_d,sl_price,type_1_M,session,initial_return,type_3_m5,type_3_m30,type_2_d,type_1_w,side,TP_distance,htf_poi_ltf_confirmation_False,htf_poi_ltf_confirmation_True,management_strategy_be_after_1r,management_strategy_be_after_push,management_strategy_no_management,management_strategy_strategy_2,outcome_loss,outcome_win
0,1,False,False,3.0,NaN,False,False,True,False,False,False,False,True,False,NaN,NaN,False,no_management,NaN,2.000000,1.1000,False,False,False,False,2025-01-02 02:24:55.600575,NaN,NaN,0.0050,False,False,False,False,False,2.000000,1.1100,False,False,1.0950,False,tokyo,0.0100,False,False,False,False,long,0.0100,True,False,False,False,True,False,False,False
1,2,False,False,2.0,loss,False,False,False,False,False,True,False,False,False,-0.0145,1.1855,False,no_management,NaN,0.666667,1.2000,False,False,False,False,2025-01-02 02:24:55.601552,NaN,NaN,0.0150,False,False,False,True,False,0.666667,1.2100,False,False,1.1850,False,tokyo,0.0100,False,False,False,False,long,0.0100,False,True,False,False,True,False,True,False
2,3,False,False,NaN,NaN,False,False,False,False,False,False,False,False,False,NaN,NaN,False,no_management,NaN,10.000000,1.3000,False,False,False,False,2024-12-27 10:00:01.000000,NaN,NaN,0.5000,False,False,False,False,False,10.000000,1.3300,False,False,1.2970,False,london,0.0300,False,False,False,False,long,0.0300,True,False,False,False,True,False,False,False
3,4,False,False,4.0,win,False,False,False,False,False,False,False,False,False,0.0050,1.3050,False,strategy_2,5.000000,10.000000,1.3000,False,False,False,False,2024-12-25 10:00:00.000000,1.3150,0.0150,0.5000,False,False,False,False,False,10.000000,1.3300,False,False,1.2970,False,london,0.0300,False,False,False,False,long,0.0300,True,False,False,False,False,True,False,True
4,5,False,False,3.0,win,False,False,False,False,False,False,False,False,False,0.0132,1.1248,False,be_after_1r,3.397959,3.000000,1.1116,False,False,False,False,2025-01-02 02:24:55.604629,1.1449,0.0333,0.0098,False,False,False,False,False,3.000000,1.1410,False,True,1.1018,False,tokyo,0.0294,False,False,True,False,long,0.0294,True,False,True,False,False,False,False,True
5,6,False,False,3.0,NaN,False,False,False,False,False,True,True,False,False,NaN,NaN,False,no_management,1.590909,3.943182,1.1213,False,False,False,False,2025-01-02 02:24:55.605254,1.1353,0.0140,0.0088,False,False,False,False,False,3.943182,1.1560,False,False,1.1125,True,tokyo,0.0347,False,False,False,False,long,0.0347,False,True,False,False,True,False,False,False
6,7,False,False,4.0,loss,False,False,False,False,False,False,False,False,False,-0.0118,1.1206,False,be_after_push,6.558824,3.323529,1.1324,False,False,False,True,2025-01-02 02:24:55.605795,1.1770,0.0446,0.0068,False,True,False,False,False,3.323529,1.1550,False,False,1.1256,False,tokyo,0.0226,False,False,False,False,long,0.0226,True,False,False,True,False,False,True,False
7,8,False,False,5.0,win,False,False,True,False,False,True,False,False,False,0.0114,1.1580,False,be_after_push,5.400000,7.927273,1.1466,False,True,False,False,2025-01-02 02:24:55.606385,1.1763,0.0297,0.0055,False,False,False,False,False,7.927273,1.1902,False,False,1.1411,False,tokyo,0.0436,False,False,False,False,long,0.0436,False,True,False,True,False,False,False,True
8,9,False,False,5.0,NaN,False,False,False,False,False,True,False,False,True,NaN,NaN,True,no_management,5.264151,8.811321,1.1381,False,False,False,False,2025-01-02 02:24:55.606952,1.1660,0.0279,0.0053,False,False,False,False,False,8.811321,1.1848,False,False,1.1328,False,tokyo,0.0467,False,F

In [153]:

df:pd.DataFrame = j.to_dataframe(config=None)
df = df.filter(regex='^trade_uid|^type|^htf|management_strat|outcome_win')
# filter for management_strategy==be_after_1r
df = df[df['management_strategy'] == 'be_after_1r']
df

TradeJournal.to_markdown

,trade_uid,type_3_m1,type_3_M,type_1_m5,type_2_m30,type_1_m1,type_3_h4,type_3_w,htf_poi_ltf_confirmation,type_2_m15,type_3_m15,type_2_h4,type_1_m30,management_strategy,type_3_d,type_2_M,type_2_m1,type_2_w,type_3_h1,type_1_h4,type_2_m5,type_2_h1,type_1_m15,type_1_h1,type_1_d,type_1_M,type_3_m5,type_3_m30,type_2_d,type_1_w,htf_poi_ltf_confirmation_False,htf_poi_ltf_confirmation_True,management_strategy_be_after_1r,management_strategy_be_after_push,management_strategy_no_management,management_strategy_strategy_2,outcome_win
4,5,False,False,False,False,False,False,False,False,False,False,False,False,be_after_1r,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,True,False,True,False,False,False,True
9,10,False,False,False,False,False,False,False,True,False,False,False,False,be_after_1r,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False
11,12,False,False,False,False,False,False,False,False,False,False,False,False,be_after_1r,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,True,False,True,False,False,False,True
